In [1]:
from manager import load_obj,save_obj
from Homework import Homework
from pprint import pprint
import re
from tqdm.notebook import tqdm
from langdetect import detect


In [2]:
##loading works from object folders
def load_works():
    works = []
    for count in range(35):
        works+= load_obj(f"objects/homeworks{count}")

    for count in range(2):
        works+= load_obj(f"objects_grin/grins{count}")
    
    return works

In [3]:
works = load_obj("works")

17848

In [10]:
## detect the language

for work in tqdm(works):
    lang=""
    work.references = "none"
    work.lang = "none"
    if len(work.text_html) > 100 :
        work.lang = detect(work.text_html)
    
    

  0%|          | 0/17848 [00:00<?, ?it/s]

In [11]:
save_obj(works, "works")


In [38]:

"""find the bilbiography section in the text and cut out text after the bilbiograpgy section (e.g. appendix)"""

def regex(text):
    text = text.lower()
    bibliography = ["literaturverzeichnis", "literatur", "quellen", "quellenverzeichnis", "bibliographie", "bibliografie", "bibliography" "referenzen", "references", "literaturhinweise", "literaturliste", "quellen und literaturverzeichnis","quellenverzeichnis und literatur", "quellen und literatur", "quellenangaben", "quellenangabe", "literaturangabe", "literaturangaben", "bilbliographien", "bilbiographien", "primärliteratur", "literarurliste", "literaturverzeihnis", "bibiografie", "bilbiographie", "verwendete literatur", "quellen- und literaturverzeichnis"]
    appendix = ["anhang", "appendix", "datensatz","zusammenfassungen", "anhänge", "anhangsverzeichnis","anhangverzeichnis","abbildungsverzeichnis", "rechtsquellenverzeichnis"]
    appendix_list=["anhangsverzeichnis","anhangverzeichnis","abbildungsverzeichnis", "rechtsquellenverzeichnis"]
    reg_word = re.compile(r'(\n.{0,11}\b(' + '|'.join(bibliography) + r')\b.{0,8}\n)') #find the bilbiography
    reg_num = re.compile(r'(\[\.\.\.\]\n\n\[1\]|\n\n1|\n\n\(1\)|\n\n\[\[1\]\])') #cut the footnotes
    reg_appendix = re.compile(r'(\n(?!.{0,8}((s)\.|siehe)).{0,8}\b(' + '|'.join(appendix) + r')\b(?!.{0,3}((s)\.|antwort|ebd\.)).{0,60}\n)') #filter out appendix
    reg_appendix_list = re.compile(r'(\n(?!.{0,8}((s)\.|siehe)).{0,8}\b(' + '|'.join(appendix_list) + r')\b.{0,60}\n)')
    
    
    match_ref = [m.end() for m in re.finditer(reg_word,text)]
    match_num = [m.start() for m in re.finditer(reg_num,text)]
    match_appendix = [m.start() for m in re.finditer(reg_appendix,text)]
    match_appendix_list = [m.start() for m in re.finditer(reg_appendix_list,text)]
    
    if match_appendix_list and match_appendix:  ### if anhangsverzeichnis befor anhang then delete anhang because anhangsverzeichnis already contains anhang 
    
        if match_appendix_list[-1] < match_appendix[-1]:
            match_appendix = match_appendix_list
    
    
    
    if match_ref and not ((len(text[match_ref[-1]:]) / len(text)) > 0.8):
   
        if match_num and (match_ref[-1] < match_num[-1]):
         
            if match_appendix and (match_ref[-1] < match_appendix[-1]):
                return[match_ref[-1],match_appendix[-1]]
            else:
                return[match_ref[-1],match_num[-1]]
        else:
            if match_appendix and (match_ref[-1] < match_appendix[-1]):
                return[match_ref[-1], match_appendix[-1]]
            else:
                return[match_ref[-1], len(text)]
        
    return []

In [39]:
#some manually removed works, which are too unformatted
wrong_one=["https://www.hausarbeiten.de/document/97247", "https://www.grin.com/document/301185","https://www.hausarbeiten.de/document/496359","https://www.grin.com/document/335877", "https://www.grin.com/document/317499", "https://www.grin.com/document/370791","https://www.hausarbeiten.de/document/107972", "https://www.hausarbeiten.de/document/496146","https://www.hausarbeiten.de/document/109917", "https://www.hausarbeiten.de/document/106770", "https://www.hausarbeiten.de/document/98212","https://www.grin.com/document/351592"]

for work in tqdm(works):
    work.only_text = "none"
    work.references = "none"
    from_to = regex(work.text_html)
    if from_to and work.lang == "de" and work.link not in wrong_one:
        work.references = work.text_html[from_to[0]:from_to[1]]
        
        new_len = len(work.references)
        index = work.text_html.find(work.references[:new_len])
        if index == -1:
            print("yes")
        work.only_text = work.text_html[:index]

  0%|          | 0/17848 [00:00<?, ?it/s]

In [40]:
"""save works as pickle object"""
save_obj = (works, "works_references")

# Some more cleaning


In [28]:
works = load_obj("works_references")

In [29]:
wordcount = 0
count = 0
for work in works:
    if work.references != "none":
        wordcount += len(work.references)
        count +=1
print(count, wordcount)

8398 22281343


In [30]:
regex = re.compile(r".+\n\s*(\[1\]).*")
for work in works:
    if work.references != "none":
        match = [m.start() for m in re.finditer(regex, work.references)]
        if match:
            work.references = work.references[0:match[0]]

In [31]:
#some special cleaning for works from the germanistik field
incomplete = ["https://www.hausarbeiten.de/document/1059953", "https://www.hausarbeiten.de/document/201597", "https://www.hausarbeiten.de/document/110647", "https://www.hausarbeiten.de/document/110399", "https://www.hausarbeiten.de/document/50662", "https://www.hausarbeiten.de/document/108475", "https://www.hausarbeiten.de/document/105886","https://www.hausarbeiten.de/document/104668", "https://www.hausarbeiten.de/document/104463", "https://www.hausarbeiten.de/document/101532","https://www.hausarbeiten.de/document/98711", "https://www.hausarbeiten.de/document/98543", "https://www.hausarbeiten.de/document/99099", "https://www.hausarbeiten.de/document/96497", "https://www.hausarbeiten.de/document/95491", "https://www.hausarbeiten.de/document/95585", "https://www.hausarbeiten.de/document/95733", "https://www.hausarbeiten.de/document/105583", "https://www.hausarbeiten.de/document/110399", "https://www.hausarbeiten.de/document/12074", "https://www.hausarbeiten.de/document/105639", "https://www.hausarbeiten.de/document/101894", "https://www.hausarbeiten.de/document/100719", "https://www.hausarbeiten.de/document/99407", "https://www.hausarbeiten.de/document/94774", "https://www.hausarbeiten.de/document/96690", "https://www.hausarbeiten.de/document/146191"]
words = ["fußnoten", "fussnoten", "anhang", r"1\. einleitung",  r"1\.1\. einleitung", r"roman\- und realismustheorie nach der revolution von 1848", r"darstellungen\:", r"\[\.\.\.\]", "staub ist eins der elemente", "die unabhängigkeitserklärung", r"über dieses dokument \.\.\."]
regex = re.compile(r".+\n("+ '|'.join(words) + r")\n")
for work in works:
    if work.link in incomplete or work.references.lower() == "abbildung in dieser leseprobe nicht enthalten\n":
        work.references = "none"
    if work.references != "none":
        match = [m.start() for m in re.finditer(regex, work.references.lower())]
        if match:
            work.references = work.references[0:match[0]]


In [32]:
for work in works:
    if work.references != "none":
        reg = re.compile(r"(^(?!(\d{4}))[^a-zA-Z]*|([pP]rimärliteratur|[sS]ekundärliteratur).{0,1}\n)", flags=re.ASCII | re.MULTILINE)
        work.references = re.sub(reg, "", work.references)
        

In [ ]:
save_obj = (works, "works_references")

In [15]:
wordcount = 0
count = 0
for work in works:
    if work.references != "none":
        wordcount += len(work.references)
        count +=1
print(count, wordcount)

8398 22281343
